In [4]:
import os
import time
from functools import reduce
import numpy as np
import pandas as pd

In [5]:
load_data = pd.read_excel("基金前十持有人.xlsx",  usecols="A,B,C,D,E,F,G,H,I,J,K,L,M")
load_data

,证券代码,证券简称,第1名持有人名称,第2名持有人名称,第3名持有人名称,第4名持有人名称,第5名持有人名称,第6名持有人名称,第7名持有人名称,第8名持有人名称,第9名持有人名称,第10名持有人名称,第11名持有人名称
0,160125.OF,南方香港优选,左群,罗远芬,夏志元,邵常红,王安民,王剑鹏,杨凯,唐亮,黄锦永,顾少华,NaN
1,165521.OF,中信保诚中证800金融A,邹建,孙毅,韦桂生,殷丽贞,叶宁,殷庆,俞加益,李锡祥,张国强,吴嫣,NaN
2,161124.OF,易方达香港恒生综合小型股A,株洲结谷实业有限责任公司,娄钧,曹莉,夏桂玉,胡欣,邓丹娃,河南百瑞工程造价咨询有限公司,赵伟,宋协锦,郑蕾,NaN
3,160416.OF,华安标普全球石油A,株洲结谷实业有限责任公司,北京新湖巨源投资管理有限公司-新湖巨源泓湖百世风险平衡1号私募证券投资基金,张汝愉,李永东,尹大力,杨岚,应宇律,潘重光,于超银,梁锟宜,NaN
4,501302.OF,南方恒生ETF联接A,朱晓颖,湛胜仁,深圳市老龄事业发展基金会,邵常红,骆念春,林小红,钱晓健,徐静凤,吴强,林淑娟,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,160219.OF,国泰国证医药卫生A,班宽,乔劲,杨碧,陈南方,梁坚勇,丁正阳,方翠云,薛智文,杨辉,林魁,NaN
1270,513200.OF,易方达中证港股通医药卫生综合ETF,百年人寿保险股份有限公司-传统保险产品,新华人寿保险股份有限公司,信泰人寿保险股份有限公司-分红产品,BARCLAYSBANKPLC,中国光大银行股份有限公司-易方达中证港股通医药卫生综合交易型开放式指数证券投资基金发起式联接基金,中国民生银行股份有限公司-汇添富添福盈和稳健养老目标一年持有期混合型基金中基金,上海浦东发展银行股份有限公司-汇添富聚焦经典一年持有期混合型基金中基金(FOF,工银安盛人寿保险有限公司,百年人寿保险股份有限公司-平衡自营,新华资管-工商银行-新华资产-明仁八号资产管理产品,易方达基金管理有限公司
1271,516820.OF,平安中证医药及医疗器械创新ETF,百年保险资管-南京银行-百年资管弘远11号资产管理产品,青岛云锦新能源科技有限公司,林生铃,李俞玮,华泰证券股份有限公司,众诚汽车保险股份有限公司-自有资金,招商证券股份有限公司,阙俊,北京华彬庄园绿色休闲健身俱乐部有限公司,张雪川,NaN
1272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# 创建空字典用于存储结果
holder_to_security_dict = {}

# 遍历 DataFrame，忽略最后一行（数据来源）
for i in range(len(load_data) - 1):
    row = load_data.iloc[i]
    security_short_name = row["证券简称"]
    holder_names = row[3:]  # 从第1名持有人名称开始

    for holder_name in holder_names:
        if pd.isna(holder_name):  # 跳过当前持有人名称，如果它是 NaN
            continue
        if len(holder_name) < 5: #过滤到个人
            continue
        if holder_name in holder_to_security_dict:
            holder_to_security_dict[holder_name].append(str(security_short_name))  # 将证券简称转换为字符串
        else:
            holder_to_security_dict[holder_name] = [str(security_short_name)]  # 将证券简称转换为字符串

data = {
    '一级分类':[],
    '二级分类':[],
    '序号':[],
    '主体': [],
    'ETF': []
}

number = 1
 
for name in holder_to_security_dict.keys():
    for value in holder_to_security_dict[name]:
        data['序号'].append(number)
        data['主体'].append(name)
        data['ETF'].append(value) 
        if "私募" in name or '量游' in name or '财富' in name : 
            data['一级分类'].append("私募基金")
            data['二级分类'].append("私募产品")
            continue  
        if "资产管理计划" in name:
            if '基金' in name: 
                data['一级分类'].append("公募基金")
                data['二级分类'].append("公募资管计划") 
            elif '期货' in name: 
                data['一级分类'].append("期货公司")
                data['二级分类'].append("期货资管计划")
            else:
                data['一级分类'].append("暂无")
                data['二级分类'].append("暂无")
            continue  
        if "基金管理有限公司" in name: 
            data['一级分类'].append("公募基金")
            data['二级分类'].append("公募基金产品")
            continue  
        if "期货" in name:
            data['一级分类'].append("期货公司")
            data['二级分类'].append("期货产品")
            continue  
        if "证券股份" in name or "证券有限公司" in name or "中国国际金融" in name or "证券有限责任公司" in name:
            data['一级分类'].append("证券公司")
            data['二级分类'].append("证券产品")
            continue
        if "保险" in name:
            data['一级分类'].append("保险")
            data['二级分类'].append("保险产品")
            continue
        if "信托" in name:
            data['一级分类'].append("信托")
            data['二级分类'].append("信托产品")
            continue
        if "理财" in name:
            data['一级分类'].append("银行")
            data['二级分类'].append("银行理财产品")
            continue
        if "联接基金" in name:
            data['一级分类'].append("公募基金")
            data['二级分类'].append("联接基金产品")
            continue
        if "养老金" in name:
            data['一级分类'].append("公募基金")
            data['二级分类'].append("养老金产品") 
            continue  
        if "FOF" in name:
            data['一级分类'].append("FOF")
            data['二级分类'].append("FOF产品") 
            continue
        else:
            data['一级分类'].append("暂无")
            data['二级分类'].append("暂无")
        
    number+=1
df = pd.DataFrame(data)
df.to_excel("ETF_all.xlsx",index=True)
    

In [7]:
# 假设您的 DataFrame 名为 load_data

# ...（之前的代码，构建 holder_to_security_dict 字典）
# ...（将字典转换为 DataFrame 的代码）

# licai_df = holder_df.loc[:, [col for col in holder_df.columns if '理财' in col]].T

# insurance_df = holder_df.loc[:, [col for col in holder_df.columns if '保险' in col]].T

# trust_df = holder_df.loc[:, [col for col in holder_df.columns if '信托' in col]].T

# security_df = holder_df.loc[:, [col for col in holder_df.columns if '证券股份' in col]].T

# MF_df = holder_df.loc[:, [col for col in holder_df.columns if '联接基金' in col]].T

# PE_df = holder_df.loc[:, [col for col in holder_df.columns if '私募' in col]].T

# futures_df = holder_df.loc[:, [col for col in holder_df.columns if '期货' in col]].T

# AM_df =  holder_df.loc[:, [col for col in holder_df.columns if '资产管理' in col and '基金' in col and '私募' not in col and ('银行' in col or '证券' in col)]].T

# pension_df = holder_df.loc[:, [col for col in holder_df.columns if '养老金' in col]].T

# FOF_df = holder_df.loc[:, [col for col in holder_df.columns if 'FOF' in col]].T

# yearfund_df = holder_df.loc[:, [col for col in holder_df.columns if '年金' in col]].T

# company_df = holder_df.loc[:, [col for col in holder_df.columns if '私募' not in col and 
# '证券' not in col and '保险' not in col and '资产管理' not in col and '年金' not in col and 
# 'FOF' not in col and '私募' not in col and '信托' not in col and '理财' not in col and
# '养老金' not in col and '投资' not in col and '银行' not in col and '资本' not in col and
# '财富' not in col and '期货' not in col and '基金' not in col and 'LOF' not in col
# ]].T

# with pd.ExcelWriter('holder_securities.xlsx') as writer:

#     holder_df.to_excel(writer, sheet_name='总表', index=False)

#     AM_df.to_excel(writer, sheet_name='公募资产专户', index=False)

#     FOF_df.to_excel(writer, sheet_name='公募FOF', index=False)

#     pension_df.to_excel(writer, sheet_name='公募养老金', index=False)

#     MF_df.to_excel(writer, sheet_name='联接基金', index=False)


#     licai_df.to_excel(writer, sheet_name='理财', index=False)

#     PE_df.to_excel(writer, sheet_name='私募', index=False)

#     trust_df.to_excel(writer, sheet_name='信托', index=False)

#     futures_df.to_excel(writer, sheet_name='期货', index=False)

#     insurance_df.to_excel(writer, sheet_name='保险', index=False)

#     security_df.to_excel(writer, sheet_name='证券', index=False)

#     yearfund_df.to_excel(writer, sheet_name='年金', index=False)

#     company_df.to_excel(writer, sheet_name='产业', index=False)

# 关闭 Excel Writer，保存文件